In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sqlalchemy import create_engine, text
!pip install PyMySql
import pymysql
import sqlalchemy

!pip install ipython-sql

#Import modules that related to SQL and connection to Azure

#Create the engine so that we can get access to Azure MySQL
username = 'A09' # Your team name
password = 'A09password'
host = 'testproject.mysql.database.azure.com'
database = 'A09'  # Your team name

connection_string = f"mysql+pymysql://{username}:{password}@{host}/{database}"

ssl_args = {
    'ssl_ca': 'path_to/ca-cert.pem',
    'ssl_cert': 'path_to/client-cert.pem',
    'ssl_key': 'path_to/client-key.pem'
}

engine = create_engine(
    connection_string,
    connect_args={'ssl': ssl_args}
)


/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00


In [2]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [3]:
data_file = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json")
data = []
for line in data_file:
  data.append(json.loads(line))
checkin_df = pd.DataFrame(data)
data_file.close()

In [4]:
data_file = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json")
data = []
for line in data_file:
  data.append(json.loads(line))
business_df = pd.DataFrame(data)
data_file.close()

In [5]:
data_file = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json")
data = []
for line in data_file:
  data.append(json.loads(line))
review_df = pd.DataFrame(data)
data_file.close()

In [6]:
data_file = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json")
data = []
for line in data_file:
  data.append(json.loads(line))
tip_df = pd.DataFrame(data)
data_file.close()

In [7]:
data_file = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json")
data = []
for line in data_file:
  data.append(json.loads(line))
user_df = pd.DataFrame(data)
data_file.close()

In [8]:
#Check if all the dataset has already been imported as DataFrames
print(type(business_df))
print(type(user_df))
print(type(checkin_df))
print(type(review_df))
print(type(tip_df))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [9]:
business_df.to_sql('business', engine, if_exists='replace', index=True, 
                   dtype={'attributes': sqlalchemy.types.JSON, 'hours': sqlalchemy.types.JSON}, chunksize = 50000)

150346

In [10]:
max_length = 255 #Set the maximum length for every element in 'date' column
checkin_df['date'] = checkin_df['date'].apply(lambda x: x[:max_length] if isinstance(x, str) else x) #Cut off all the characters that is over 255 limit
user_df['friends'] = user_df['friends'].apply(lambda x: x[:max_length] if isinstance(x, str) else x) 

In [11]:
checkin_df.to_sql('checkin', engine, if_exists='replace', index = True, chunksize = 50000)

131930

In [12]:
tip_df.to_sql('tip', engine, if_exists='replace', index=True, chunksize = 50000)

908915

In [13]:
user_df.to_sql('user', engine, if_exists='replace', index=True, chunksize = 50000)

1987897

In [14]:
review_df.iloc[:1000].to_sql('review', engine, if_exists='replace', index=True, chunksize = 50000)

1000

In [15]:
query = "SELECT * FROM review where stars=3.0 LIMIT 5;"

# Use the engine to execute the query and load the result into a DataFrame
result_df = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df

,index,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
2,8,ZKvDG2sBvHVdF5oBNUOpAQ,wSTuiTk-sKNdcFyprzZAjg,B5XSoSG3SfvQGtKEGQ1tSQ,3.0,1,1,0,This easter instead of going to Lopez Lake we ...,2016-03-30 22:46:33
3,9,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06
4,21,ZVvhc3Go7v5I8XTiVoWmqQ,WBpQDAZymU0dhIqXcACGNw,ut6fi2W2YaipNOqvi7e0jw,3.0,0,0,0,"Upland is a brewery based out of Bloomington, ...",2014-11-12 14:12:20
